In [ ]:
import websockets
import json
from datetime import datetime, timezone
from time import time as perf_counter
import sys
import os

project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

from config import AIS_API_KEY, AIS_STREAM_URL, AIS_BOUNDING_BOXES


In [ ]:
async def connect_ais_stream():

    ship_names = {}
    
    async with websockets.connect(AIS_STREAM_URL) as websocket:
        subscribe_message = {
            "APIKey": AIS_API_KEY, 
            "BoundingBoxes": AIS_BOUNDING_BOXES
        }
        
        subscribe_message_json = json.dumps(subscribe_message)
        await websocket.send(subscribe_message_json)
        
        message_count = 0
        
        async for message_json in websocket:
            message = json.loads(message_json)
            message_type = message["MessageType"]
            
            if message_type == "ShipStaticData":
                static_data = message.get('Message', {}).get('ShipStaticData', {})
                ship_id = static_data.get('UserID')
                ship_name = static_data.get('Name', '').strip()
                if ship_id and ship_name:
                    was_unknown = ship_id not in ship_names
                    ship_names[ship_id] = ship_name
                    if was_unknown:
                        print(f"Got name for ShipID {ship_id}: {ship_name}")
            
            if message_type == "PositionReport":
                ais_message = message.get('Message', {}).get('PositionReport', {})
                
                if ais_message:
                    message_count += 1
                    
                    ship_id = ais_message.get('UserID', 'N/A')
                    lat = ais_message.get('Latitude', 0)
                    lon = ais_message.get('Longitude', 0)
                    
                    speed = ais_message.get('Sog', None)
                    course = ais_message.get('Cog', None)
                    true_heading = ais_message.get('TrueHeading', None)
                    
                    heading = None if (true_heading is None or true_heading == 511) else true_heading
                    
                    ship_name = ship_names.get(ship_id, 'Unknown')
                    
                    # time_str = datetime.now(timezone.utc).strftime("%H:%M:%S")
                    
                    info_parts = []
                    if speed is not None and speed > 0:
                        info_parts.append(f"Speed: {speed:.1f} kn")
                    if course is not None:
                        info_parts.append(f"Course: {course:.1f}°")
                    if heading is not None:
                        info_parts.append(f"Heading: {heading:.1f}°")
                    
                    info_str = " | ".join(info_parts) if info_parts else ""
                    
                    print(f"[{time_str}] #{message_count:4d} | ShipID: {ship_id:12d} | "
                          f"Name: {ship_name:20s} | "
                          f"Lat: {lat:8.5f}° | Lon: {lon:9.5f}°" + 
                          (f" | {info_str}" if info_str else ""))

await connect_ais_stream()

[15:25:44] #   1 | ShipID:    372594000 | Name: Unknown              | Lat: 13.68784° | Lon: 100.55418° | Course: 0.0° | Heading: 192.0°
Got name for ShipID 525012169: TB CLEMENT 1
Got name for ShipID 563105900: PSA MARVEL CS05
[15:25:44] #   2 | ShipID:    564275000 | Name: Unknown              | Lat:  1.21439° | Lon: 103.88428° | Speed: 8.4 kn | Course: 242.3° | Heading: 244.0°
[15:25:45] #   3 | ShipID:    477997211 | Name: Unknown              | Lat: 22.28779° | Lon: 114.15967° | Speed: 0.1 kn | Course: 339.0° | Heading: 197.0°
[15:25:45] #   4 | ShipID:    563086930 | Name: Unknown              | Lat:  1.24422° | Lon: 103.75388° | Speed: 11.3 kn | Course: 56.4° | Heading: 63.0°
[15:25:45] #   5 | ShipID:    525501277 | Name: Unknown              | Lat: -7.16884° | Lon: 112.69163° | Course: 0.0°
[15:25:45] #   6 | ShipID:    352006352 | Name: Unknown              | Lat:  1.27510° | Lon: 103.93379° | Course: 284.6° | Heading: 43.0°
[15:25:45] #   7 | ShipID:    564743000 | Name: Unk

CancelledError: 